# Speckle Federator

The purpose of this notebook is to take 3 commits, join them into a new 4th commit and then use the embedded viewer to benchmark performance relative to overlaying them.

In [1]:
%%capture
# capture turns off the output for this cell which would just be the pip install log 🤫
%poetry install specklepy

%poetry install python-dotenv

%reload_ext dotenv
%dotenv

In [2]:
import os
HOST_SERVER = os.getenv('XYZ_HOST_SERVER')
ACCESS_TOKEN = os.getenv('XYZ_ACCESS_TOKEN')

In [3]:
from specklepy.api.client import SpeckleClient

client = SpeckleClient(host=HOST_SERVER)  # or whatever your host is
client.authenticate_with_token(ACCESS_TOKEN)  # or whatever your token is

client

SpeckleClient( server: https://speckle.xyz, authenticated: True )

In [4]:
# Define 3 commits in a single stream.add()
commits = [
    "https://speckle.xyz/streams/7ce9010d71/commits/27a5df66a0",  # stadium seating
    "https://speckle.xyz/streams/7ce9010d71/commits/f7ef7f5270",  # stadium roof
    "https://speckle.xyz/streams/7ce9010d71/commits/f6052eaa16",  # stadium facade
]

## Benchmark 1:
Generate an overlay URL that combines the 3 commits

`\streams\{STREAM_ID}\commits\{COMMIT_ID1}?overlay={COMMIT_ID2},{COMMIT_ID3}`

In [5]:
from specklepy.transports.server import ServerTransport
from specklepy.api.wrapper import StreamWrapper

wrappers = [StreamWrapper(commit_url) for commit_url in commits]

transport = ServerTransport(client=client, stream_id=wrappers[0].stream_id)

In [6]:
stream_id = wrappers[0].stream_id

commit_ids = [wrapper.commit_id for wrapper in wrappers]

transparency = True  # make the background transparent
autoload = True  # load the stream on page load
hide_controls = False  # hide the viewer tools button
hide_sidebar = True  # hide the sidebar
hide_selection_info = True  # hide the selection info

# url = https://speckle.xyz/embed?stream=ca99defd4b&commit=39009883ba&transparent=true&autoload=true&hidecontrols=true&hidesidebar=true&hideselectioninfo=true
embed_url = f"https://speckle.xyz/embed?stream={stream_id}&commit={commit_ids[0]}&overlay={','.join(commit_ids[1:])}&transparent={transparency}&autoload={autoload}&hidecontrols={hide_controls}&hidesidebar={hide_sidebar}&hideselectioninfo={hide_selection_info}"

In [7]:
from IPython.display import IFrame

IFrame(embed_url, width=400, height=300)

## Benchmark 2: Combine commits

Take the content of the 3 commits and commit that to speckle.

View that commit in the embedded viewer

In [8]:
commit_objects = [client.commit.get(stream_id, commit_id) for commit_id in commit_ids]

f"{commit_objects[0].authorName}: {commit_objects[0].message} on {commit_objects[0].createdAt}"

'Jonathon Broughton: Sent 197 elements from Navisworks. on 2022-12-30 23:26:33.611000+00:00'

If the 3 urls given are all commits then the first name object has a property called `referencedObject` which is the id of the wrapper object for the commit data.

In [26]:
referenced_objects = [r.referencedObject for r in commit_objects]

We can create a new Federation class which is essentially just adding a name for the collection.

In [10]:
from specklepy.objects import Base


class Federation(Base, speckle_type="Federation"):
    def __init__(self, **kwargs):
      self["Components"] = []

In [27]:
new_commit_object = Base(speckle_type="Federation")

new_commit_object["Components"] = [
    Base.of_type("reference", referencedId=commit_id)
    for commit_id in referenced_objects
]

For the Viewer to resolve this will require the closure table for each of the reference objects. These closures are used as a shortcut for it to handle the processable objects.

We can add a custom operation to our notebook to simply get this from the server.

Ideally we'd check the localTransport first to see if we have the closure table already, but for brevity we'll just get it from the server.

We'll make a straight GraphQL query of the commit


In [19]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


def get_closures(wrapper, object_id):
    client = Client(
        transport=RequestsHTTPTransport(
            url=f"{wrapper._account.serverInfo.url}/graphql", verify=True, retries=3
        )
    )

    query = gql(
        """ query Object($stream_id: String!, $object_id: String!) { 
            stream(id: $stream_id) { 
              object(id: $object_id) { 
                data 
              }
            }
          } """
    )
    params = {"stream_id": wrapper.stream_id, "object_id": object_id}
    return client.execute(query, variable_values=params)["stream"]["object"]["data"][
        "__closure"
    ]

The `new_commit_object` will need the `__closure` table from each of the commits were merging. We can use the `get_closures` function we created earlier to get this.

At this point we could refactor to always be using Lists rather than numbered variables, but for now we'll just add the closures to the new commit object.

In [28]:
closures = {
    k: v
    for d in [get_closures(wrappers[0], obj_id) for obj_id in referenced_objects]
    for k, v in d.items()
}
closures.update(dict.fromkeys(referenced_objects, 1))


new_commit_object["__closure"] = closures

Within a notebook it often helps to write defensive code that can be run repeatedly even if the method has already been run. This is because the notebook is stateful and can be run in any order. The following code checks if the `federated` branch exists and if not creates it.

In [14]:
from specklepy.transports.server import ServerTransport
from specklepy.api import operations
from specklepy.logging.exceptions import GraphQLException

def try_get_branch_or_create(client, stream_id, branch_name):
    try:
        client.branch.get(
            stream_id=stream_id, name=branch_name
        ) or client.branch.create(stream_id=stream_id, name=branch_name)
        return client.branch.get(
            stream_id=stream_id, name=branch_name)
    except GraphQLException:
        return client.branch.create(stream_id=stream_id, name=branch_name)


branch = try_get_branch_or_create(client, stream_id, "federated")

In [15]:
hash_2 = operations.send(base=new_commit_object, transports=[transport])

hash_2

'e654f22a34f02c5c6d8e34e0d6cf01e1'

This doesn't actually work as the default specklepy traversal strips props with __ prefix and neither does it resolve closure for Reference Objects. So we'll need to add a custom operation to the server to resolve this.

In [21]:
from typing import Any, Dict, List, Optional, Tuple
from specklepy.serialization.base_object_serializer import BaseObjectSerializer
from uuid import uuid4
import hashlib
import re
from enum import Enum
from specklepy.objects.base import Base, DataChunk
import ujson

PRIMITIVES = (int, float, str, bool)


def traverse_base(
    serializer: BaseObjectSerializer, base: Base, closures: Dict[str, Any] = {}
):
    if serializer.write_transports:
        for wt in serializer.write_transports:
            wt.begin_write()

    if not serializer.detach_lineage:
        serializer.detach_lineage = [True]

        serializer.lineage.append(uuid4().hex)
        object_builder = {"id": "", "speckle_type": "Base", "totalChildrenCount": 0}
        object_builder.update(speckle_type=base.speckle_type)
        obj, props = base, base.get_serializable_attributes()

        while props:
            prop = props.pop(0)
            value = getattr(obj, prop, None)
            chunkable = False
            detach = False

            # skip props marked to be ignored with "__" or "_"
            if prop.startswith(("__", "_")):
                continue

            # don't prepopulate id as this will mess up hashing
            if prop == "id":
                continue

            # only bother with chunking and detaching if there is a write transport
            if serializer.write_transports:
                dynamic_chunk_match = prop.startswith("@") and re.match(
                    r"^@\((\d*)\)", prop
                )
                if dynamic_chunk_match:
                    chunk_size = dynamic_chunk_match.groups()[0]
                    serializer._chunkable[prop] = (
                        int(chunk_size) if chunk_size else base._chunk_size_default
                    )

                chunkable = prop in base._chunkable
                detach = bool(
                    prop.startswith("@") or prop in base._detachable or chunkable
                )

            # 1. handle None and primitives (ints, floats, strings, and bools)
            if value is None or isinstance(value, PRIMITIVES):
                object_builder[prop] = value
                continue

            # NOTE: for dynamic props, this won't be re-serialised as an enum but as an int
            if isinstance(value, Enum):
                object_builder[prop] = value.value
                continue

            # 2. handle Base objects
            elif isinstance(value, Base):
                child_obj = serializer.traverse_value(value, detach=detach)
                if detach and serializer.write_transports:
                    ref_id = child_obj["id"]
                    object_builder[prop] = serializer.detach_helper(ref_id=ref_id)
                else:
                    object_builder[prop] = child_obj

            # 3. handle chunkable props
            elif chunkable and serializer.write_transports:
                chunks = []
                max_size = base._chunkable[prop]
                chunk = DataChunk()
                for count, item in enumerate(value):
                    if count and count % max_size == 0:
                        chunks.append(chunk)
                        chunk = DataChunk()
                    chunk.data.append(item)
                chunks.append(chunk)

                chunk_refs = []
                for c in chunks:
                    serializer.detach_lineage.append(detach)
                    ref_id, _ = serializer._traverse_base(c)
                    ref_obj = serializer.detach_helper(ref_id=ref_id)
                    chunk_refs.append(ref_obj)
                object_builder[prop] = chunk_refs

            # 4. handle all other cases
            else:
                child_obj = serializer.traverse_value(value, detach)
                object_builder[prop] = child_obj

            closure = {}
            # add closures & children count to the object
            detached = serializer.detach_lineage.pop()
            if serializer.lineage[-1] in serializer.family_tree:
                closure = {
                    ref: depth - len(serializer.detach_lineage)
                    for ref, depth in serializer.family_tree[
                        serializer.lineage[-1]
                    ].items()
                }

            ############ ADDING OUR MAGIC HERE #################################
            closure.update(closures)

            object_builder["totalChildrenCount"] = len(closure)

            obj_id = hashlib.sha256(ujson.dumps(object_builder).encode()).hexdigest()[
                :32
            ]

            object_builder["id"] = obj_id
            if closure:
                object_builder["__closure"] = serializer.closure_table[obj_id] = closure

            # write detached or root objects to transports
            if detached and serializer.write_transports:
                for t in serializer.write_transports:
                    t.save_object(
                        id=obj_id, serialized_object=ujson.dumps(object_builder)
                    )

            del serializer.lineage[-1]

            if serializer.write_transports:
                for wt in serializer.write_transports:
                    wt.end_write()

            return obj_id, object_builder

In [22]:
serializer = BaseObjectSerializer(write_transports=[transport])

obj_id, serialized_object = traverse_base(serializer, new_commit_object, closures)

In [23]:
commit_id2 = client.commit.create(
    branch_name=branch.name,
    stream_id=stream_id,
    object_id=obj_id,
    message="federated commit",
)

Once again we build the embed URL and display it.

In [24]:
embed_url2 = f"https://speckle.xyz/embed?stream={stream_id}&commit={commit_id2}&transparent={transparency}&autoload={autoload}&hidecontrols={hide_controls}&hidesidebar={hide_sidebar}&hideselectioninfo={hide_selection_info}"

from IPython.display import IFrame

IFrame(embed_url2, width=400, height=300)

In [25]:
embed_url2

'https://speckle.xyz/embed?stream=7ce9010d71&commit=9f68a5fd17&transparent=True&autoload=True&hidecontrols=False&hidesidebar=True&hideselectioninfo=True'

This federation is kinda simple, kinda clunky and doesn't de-dupe at all as it is not even examining the content of the individual commits.

To do anything approaching this we need to:

- load the child members of each commit
- have a strategy for de-duping
- have a strategy for merging
- have a strategy for handling any other conflicts



## Benchmark 3: Combine commits and de-dupe

The next step is to extract the component objects from each commit and then de-dupe them. This is a bit more involved as we need to load the objects from the server and then compare them. We also need to be able to merge them.

In [30]:
commit_data = [
    operations.receive(obj_id=ref_obj, remote_transport=wrap.get_transport())
    for ref_obj, wrap in zip(referenced_objects, wrappers)
]

We create 3 Base objects and create a commit essentially identical to Benchmark 2.

In [37]:
another_commit_object = Base(speckle_type="Federation.Granular")
another_commit_object["@Components"] = commit_data

... and hash them

In [38]:
hash3 = operations.send(base=another_commit_object, transports=[transport])

And then we create a new commit that contains the resolved objects

In [39]:
commit_id3 = client.commit.create(

    branch_name=branch.name,
    stream_id=stream_id,
    object_id=hash3,
    message="federated commit",
)

embed_url3 = f"https://speckle.xyz/embed?stream={stream_id}&commit={commit_id3}&transparent={transparency}&autoload={autoload}&hidecontrols={hide_controls}&hidesidebar={hide_sidebar}&hideselectioninfo={hide_selection_info}"

from IPython.display import IFrame

IFrame(embed_url3, width=400, height=300)

In [40]:
embed_url3

'https://speckle.xyz/embed?stream=7ce9010d71&commit=8671501e17&transparent=True&autoload=True&hidecontrols=False&hidesidebar=True&hideselectioninfo=True'